In [1]:
from air_hockey_challenge.framework.air_hockey_challenge_wrapper import AirHockeyChallengeWrapper
from air_hockey_challenge.framework.challenge_core import ChallengeCore
from air_hockey_challenge.framework.agent_base import AgentBase
from examples.control.hitting_agent import build_agent, HittingAgent
from examples.control.hitting_agent_wait import HittingAgentWait

from mushroom_rl.utils.dataset import parse_dataset, select_random_samples
from mushroom_rl.policy import GaussianTorchPolicy

import torch
import torch.nn as nn

from tqdm import tqdm

import pickle

import numpy as np

use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'
print(f"Cuda: {use_cuda}")


Cuda: False


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

class MushroomRLTrajectoryDataset(Dataset):
    def __init__(self, mdp, agent, n_episodes):
        """
        Initialize the MushroomRLTrajectoryDataset.

        Args:
            mdp (mushroom_rl.environments.Environment): the MDP (Markov Decision Process).
            agent: the agent to evaluate.
        """
        self.core = ChallengeCore(agent, mdp)
                # Initialize empty lists for each data type
        state_list, action_list, reward_list, next_state_list, absorbing_list, last_list = [], [], [], [], [], []

        for i in tqdm(range(n_episodes)):
            # Evaluate one episode at a time
            trajectory = self.core.evaluate(n_episodes=1, render=False)
            state, action, reward, next_state, absorbing, last = parse_dataset(trajectory)

            # Append the data from the current episode
            state_list.append(torch.from_numpy(state).to(device))
            action_list.append(torch.from_numpy(action).to(device))
            reward_list.append(torch.from_numpy(reward).to(device))
            next_state_list.append(torch.from_numpy(next_state).to(device))
            absorbing_list.append(torch.from_numpy(absorbing).to(device))
            last_list.append(torch.from_numpy(last).to(device))

        # Concatenate the data from all episodes
        self.state = torch.cat(state_list, dim=0)
        self.action = torch.cat(action_list, dim=0)
        self.reward = torch.cat(reward_list, dim=0)
        self.next_state = torch.cat(next_state_list, dim=0)
        self.absorbing = torch.cat(absorbing_list, dim=0)
        self.last = torch.cat(last_list, dim=0)

        self.length = self.state.shape[0]

    def __len__(self):
        """
        Return the total number of state-action-reward-next_state tuples in the dataset.

        Returns:
            int: the length of the dataset.
        """
        return self.length

    def __getitem__(self, index):
        """
        Get the state-action-reward-next_state tuple at the specified index.

        Args:
            index (int): the index of the desired tuple.

        Returns:
            dict: a dictionary containing state, action, reward, and next_state.
        """

        return {
            'state': self.state[index],
            'action': self.action[index],
            'reward': self.reward[index],
            'next_state': self.next_state[index],
            'absorbing': self.absorbing[index],
            'last': self.last[index],
        }



In [3]:
class Network(nn.Module):
    def __init__(self, input_shape, output_shape, n_features, **kwargs):
        super(Network, self).__init__()

        self._h1 = nn.Linear(input_shape[0], n_features)
        self._h2 = nn.Linear(n_features, n_features)
        self._h3 = nn.Linear(n_features, output_shape[0])

        nn.init.xavier_uniform_(self._h1.weight,
                                gain=nn.init.calculate_gain('tanh'))
        nn.init.xavier_uniform_(self._h2.weight,
                                gain=nn.init.calculate_gain('tanh'))
        nn.init.xavier_uniform_(self._h3.weight,
                                gain=nn.init.calculate_gain('linear'))

    def forward(self, obs, **kwargs):
        features1 = torch.tanh(self._h1(torch.squeeze(obs, 1).float()))
        features2 = torch.tanh(self._h2(features1))
        return self._h3(features2)


class BCAgent(AgentBase):
    def __init__(self, env_info, policy, **kwargs):
        super().__init__(env_info, **kwargs)
        self.policy = policy

    def reset(self):
        pass

    def draw_action(self, observation):
        return self.policy.draw_action(observation).reshape(2,3)


In [19]:
def train_dagger_agent(learner_policy, expert_policy, mdp, dataset, num_iterations=10, batch_size=64, lr=0.001):
    # Create data loader
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Set up the optimizer and loss function
    optimizer = optim.Adam(learner_policy.parameters(), lr=lr)

    # Train the initial policy on collected data
    for epoch in range(num_iterations):
        for i, data in enumerate(data_loader):
            optimizer.zero_grad()

            states = data['state'].float()

            # Get expert actions for each state in the batch
            expert_actions = []
            successful_states = []

            for state in states:
                expert_policy.reset()
                state_np = state.cpu().numpy()

                expert_action = expert_policy.draw_action(state_np)

                if not expert_policy.optimization_failed:
                    successful_states.append(state)

                    expert_actions.append(torch.from_numpy(expert_action))

            successful_states = torch.stack(successful_states)
            expert_actions = torch.stack(expert_actions).reshape((-1, 6))
            print(f"successful states: {successful_states}")
            print(f"expert actions: {expert_actions}")
            print(f"expert actions shape: {expert_actions.shape}")
            print(f"successful states shape: {successful_states.shape}")

            # Compute loss
            loss = -policy.log_prob_t(successful_states, expert_actions).mean()

            # Update the learner policy
            loss.backward()
            optimizer.step()

        # Evaluate the current learner policy
        learner_trajectory_dataset = MushroomRLTrajectoryDataset(mdp, BCAgent(mdp.env_info, learner_policy), n_episodes=2)
        data_loader = DataLoader(learner_trajectory_dataset, batch_size=batch_size, shuffle=True)

        # Collect new expert actions for the new states
        for state in learner_trajectory_dataset:
            state_np = state['state'].float().unsqueeze(0).cpu().numpy()
            expert_action = expert_policy.draw_action(state_np).astype(np.float32)
            state['action'] = torch.from_numpy(expert_action).squeeze().tolist()

        # Add the new data to the dataset
        dataset.trajectories.extend(learner_trajectory_dataset.trajectories)
        dataset.length += learner_trajectory_dataset.length

    return learner_policy


# 1. defining BCAgent and expert agent to be trained in dagger

In [5]:
env = "3dof-hit"

mdp = AirHockeyChallengeWrapper(env)
mdp.reset()

# policy can only output 1D actions (6,) ... they need to be recast in (2,3) shape later on
policy = GaussianTorchPolicy(Network, (12,), (6,), std_0=1., n_features=64, use_cuda=use_cuda)

policy = policy.load('dataset/hit_500_policy')
bc_agent = BCAgent(mdp.env_info, policy)
dataset = MushroomRLTrajectoryDataset(mdp, bc_agent, n_episodes=10)


expert_agent = HittingAgentWait(mdp.env_info)


100%|██████████| 10/10 [00:32<00:00,  3.22s/it]A


In [20]:
dagger_agent = train_dagger_agent(policy, expert_agent, mdp, dataset, num_iterations=1, batch_size=64, lr=0.001)

successful states: tensor([[ 0.8765,  0.0355,  0.0000,  0.0000,  0.0000,  0.0000, -1.1016,  1.2411,
          1.3769, -0.0406,  1.2615, -6.6864],
        [ 0.9048,  0.3271,  0.0000,  0.0000,  0.0000,  0.0000, -1.0110,  1.0760,
          0.9594,  0.3649, -0.8919,  0.4557],
        [ 1.2056,  0.1764,  0.0000,  0.0000,  0.0000,  0.0000, -1.0060,  1.0279,
          1.3844,  0.7014, -0.6074, -5.7935],
        [ 0.9048,  0.3271,  0.0000,  0.0000,  0.0000,  0.0000, -0.6559,  0.6832,
          1.2903,  0.4457,  0.3626,  1.0613]])
expert actions: tensor([[-1.1045,  1.2578,  1.3055, -0.2475,  0.4063, -0.4574],
        [-1.0064,  1.0650,  0.9664,  0.0952, -0.2052,  0.2534],
        [-1.0003,  1.0243,  1.3239, -0.1181,  0.2116, -0.2241],
        [-0.6545,  0.6937,  1.2947, -0.3136,  0.6982, -0.6215]])
expert actions shape: torch.Size([4, 6])
successful states shape: torch.Size([4, 12])


Exception in thread Thread-14044:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 86, in _plan_trajectory_thread
    _, joint_pos_traj = self.optimizer.optimize_trajectory(ee_traj, joint_pos, joint_vel, q_anchor)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/baseline/baseline_agent/optimizer.py", line 35, in optimize_trajectory
    success, dq_next = self._solve_aqp(des_point[:3], q_cur, dq_anchor)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/baseline/baseline_agent/optimizer.py", line 63, in _solve_aqp
    solver.setup(P=sparse.csc_matrix(P), q=q, A=sparse.csc_matrix(A), l=l, u=u, verbose=False

ERROR in validate_data: Lower bound at index 2 is greater than upper bound: 3.4265e+00 > 1.4364e+00
ERROR in osqp_setup: Problem data validation.
successful states: tensor([[ 1.2927e+00,  2.0504e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -7.3053e-01,  5.3752e-01,  7.0488e-01, -5.7626e-01,
          2.6118e+00, -7.3204e+00],
        [ 1.9817e+00, -4.0271e-01,  1.0411e+00, -6.9680e-02,  4.6303e-02,
          2.0197e+00,  1.4227e-01, -8.7709e-01, -2.0052e+00, -2.3426e-01,
         -7.3116e-01, -6.5113e+00],
        [ 1.0602e+00, -2.9419e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -1.0048e+00,  1.1381e+00,  1.1063e+00,  2.0726e+00,
         -4.4409e+00,  3.9288e+00],
        [ 1.0602e+00, -2.9419e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -1.0539e+00,  1.2420e+00,  1.0135e+00,  1.6063e+00,
         -2.9155e+00,  1.5912e+00],
        [ 1.2207e+00, -1.9818e-01, -1.5026e-04,  2.8333e+00, -1.9533e+00,
         -1.9625e-03, -8.

Exception in thread Thread-14258:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 85, in _plan_trajectory_thread
    ee_traj, hit_idx, q_anchor = self.plan_ee_trajectory(puck_pos, ee_pos)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 116, in plan_ee_trajectory
    res = np.array([self.bezier_planner.get_point(t_i) for t_i in np.arange(0, self.bezier_planner.t_final + 1e-6,
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 116, in <listcomp>
    res = np.array([self.bezier_planner.get_point(t_i) for t_i in np.a

successful states: tensor([[ 1.2056,  0.1764,  0.0000,  0.0000,  0.0000,  0.0000, -0.7313,  0.6481,
          0.8347,  3.0138, -8.1607,  8.9815],
        [ 1.2927,  0.2050,  0.0000,  0.0000,  0.0000,  0.0000, -1.0685,  1.1300,
          1.4436,  1.4320, -2.5474, -0.4850],
        [ 0.9048,  0.3271,  0.0000,  0.0000,  0.0000,  0.0000, -0.9755,  1.0082,
          1.2509,  0.0424, -0.7991,  1.4941],
        [ 2.0027, -0.4166,  0.4428, -0.0801,  0.0533,  2.2590,  0.3799, -0.6689,
         -1.5559, -1.4207, -0.8794, -0.6162],
        [ 1.0602, -0.2942,  0.0000,  0.0000,  0.0000,  0.0000,  0.5688, -0.7005,
         -0.8718,  2.5822, -7.3779, 11.4217],
        [ 1.0302,  0.0196,  0.0000,  0.0000,  0.0000,  0.0000, -0.7114,  0.6359,
          0.8402,  1.1990, -2.4023,  2.4565],
        [ 1.2927,  0.2050,  0.0000,  0.0000,  0.0000,  0.0000, -0.1590, -0.0975,
          0.8611,  0.6129,  1.9707, -4.8866],
        [ 1.2927,  0.2050,  0.0000,  0.0000,  0.0000,  0.0000, -0.1733, -0.1260,
          0

...

successful states: tensor([[ 1.0602e+00, -2.9419e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  6.4362e-01, -9.3698e-01, -3.0605e-01, -9.4824e-01,
          4.2531e+00, -5.7675e+00],
        [ 1.2886e+00,  2.5110e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -1.1115e+00,  1.3339e+00,  1.0527e+00,  1.1179e+00,
         -1.3193e+00,  3.0406e-01],
        [ 1.9964e+00, -4.1245e-01,  6.2061e-01, -7.7006e-02,  5.1172e-02,
          2.1879e+00,  2.3398e-01, -6.2297e-01, -1.6427e+00, -8.8438e-01,
         -1.5499e+00,  2.5061e-01],
        [ 9.8584e-01,  3.8181e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -1.1339e+00,  1.2626e+00,  1.4555e+00,  3.2963e-01,
         -6.1771e-01, -1.3599e-01],
        [ 1.2927e+00,  2.0504e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -6.3976e-01,  2.3636e-01,  1.3230e+00, -3.3564e-01,
          4.4576e-01, -9.9846e-01],
        [ 9.0482e-01,  3.2709e-01,  0.0000e+00,  0.0000e+00, 

Exception in thread Thread-17058:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 85, in _plan_trajectory_thread
    ee_traj, hit_idx, q_anchor = self.plan_ee_trajectory(puck_pos, ee_pos)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 122, in plan_ee_trajectory
    p = np.hstack([p, np.ones((p.shape[0], 1)) * self.ee_height])
  File "<__array_function__ internals>", line 200, in hstack
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/site-packages/numpy/core/shape_base.py", line 368, in hstack
    return _nx.concatenate(arrs, 0, dtype=dtype, casting=castin

successful states: tensor([[ 1.9979, -0.4135,  0.5767, -0.0778,  0.0517,  2.2054,  0.2788, -0.6548,
         -1.5907, -1.9266,  0.9168, -2.4920],
        [ 1.6782,  0.4287,  0.8315,  0.7770, -0.6016,  8.4450, -0.2848,  0.4639,
          0.4512,  2.0301, -6.5421,  5.9445],
        [ 1.0602, -0.2942,  0.0000,  0.0000,  0.0000,  0.0000, -1.1451,  1.2821,
          1.4502,  0.1966, -0.3076,  0.0409],
        [ 1.0602, -0.2942,  0.0000,  0.0000,  0.0000,  0.0000, -1.1374,  1.3272,
          1.2187,  0.0569,  1.1062, -4.6527],
        [ 0.9048,  0.3271,  0.0000,  0.0000,  0.0000,  0.0000, -0.9696,  0.9734,
          1.3042, -0.5275,  0.7966, -2.0930],
        [ 1.2927,  0.2050,  0.0000,  0.0000,  0.0000,  0.0000, -1.0412,  1.1228,
          1.2422,  0.3685,  0.6668, -6.3240]])
expert actions: tensor([[ 0.2788, -0.6548, -1.5907,  0.0000,  0.0000,  0.0000],
        [-0.2848,  0.4639,  0.4512,  0.0000,  0.0000,  0.0000],
        [-1.1433,  1.2792,  1.4503, -0.0211,  0.0221, -0.0276],
        [-

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.41s/it]

 50%|█████     | 1/2 [00:04<00:04,  4.42s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.40s/it]

100%|██████████| 2/2 [00:08<00:00,  4.42s/it]


AttributeError: 'MushroomRLTrajectoryDataset' object has no attribute 'trajectories'

# testing expert agent

In [9]:
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

for i, data in enumerate(data_loader):
    states = data['state'].float()
    for state in states:
        expert_agent.reset()
        state_np = state.cpu().numpy()
        expert_action = expert_agent.draw_action(state_np)
        print(f"expert_action: {expert_action}")
        print(f"optimization_failed: {expert_agent.optimization_failed}")
    # expert_actions = expert_agent.draw_action(states).float()

expert_action: [[-0.00474991  0.35589615 -0.5072534 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[ 0.18190074 -0.4883073   0.8293152 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.6237628   0.49058458  0.4572864 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.6604817   0.5212704   0.969205  ]
 [-0.15149744  0.37543228 -0.3254726 ]]
optimization_failed: False
expert_action: [[-0.3713106   0.5788531   0.14347985]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.8400942   0.57402956 -0.5160753 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[0.18532197 0.13293336 0.03667677]
 [0.         0.         0.        ]]
optimization_failed: True
expert_action: [[-0.10467133 -0.07287078  0.24252151]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.2975505   0.21409406  1.0

Exception in thread Thread-4018:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 85, in _plan_trajectory_thread
    ee_traj, hit_idx, q_anchor = self.plan_ee_trajectory(puck_pos, ee_pos)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 116, in plan_ee_trajectory
    res = np.array([self.bezier_planner.get_point(t_i) for t_i in np.arange(0, self.bezier_planner.t_final + 1e-6,
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 116, in <listcomp>
    res = np.array([self.bezier_planner.get_point(t_i) for t_i in np.ar

expert_action: [[ 0.15865289 -0.44082004  0.2523563 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[ 0.04151758 -0.13285692  0.57445925]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-1.259395    0.50566864 -0.5159055 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.8010943  0.8151319  0.8561258]
 [ 0.         0.         0.       ]]
optimization_failed: False
expert_action: [[-0.03954968 -0.09363928  0.8600023 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.08591827  0.06232821  0.4467567 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[ 0.39174122 -0.61539084 -0.31317687]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[ 0.16239703 -0.4859524   0.29144177]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[ 0.2058224  -0.30752552 -0.0

Exception in thread Thread-4255:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 86, in _plan_trajectory_thread
    _, joint_pos_traj = self.optimizer.optimize_trajectory(ee_traj, joint_pos, joint_vel, q_anchor)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/baseline/baseline_agent/optimizer.py", line 35, in optimize_trajectory
    success, dq_next = self._solve_aqp(des_point[:3], q_cur, dq_anchor)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/baseline/baseline_agent/optimizer.py", line 63, in _solve_aqp
    solver.setup(P=sparse.csc_matrix(P), q=q, A=sparse.csc_matrix(A), l=l, u=u, verbose=False,

expert_action: [[-0.44493315  0.50703895  1.028452  ]
 [-0.38755116  0.92173886 -0.75705886]]
optimization_failed: False
expert_action: [[-0.04764035  0.5493459  -0.613633  ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.2871652   0.17682706  0.992462  ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.9479595   1.0538996   0.71533483]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[ 0.05045802  0.5396129  -0.77464026]
 [ 0.          0.          0.        ]]
optimization_failed: True
ERROR in validate_data: Lower bound at index 2 is greater than upper bound: 3.4265e+00 > 1.4364e+00
ERROR in osqp_setup: Problem data validation.
expert_action: [[ 0.14226988 -0.87709254 -2.005182  ]
 [ 0.          0.          0.        ]]
optimization_failed: False
expert_action: [[ 0.07306751  0.39050758 -0.10339569]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_actio

...

expert_action: [[-1.1181271   1.2651008   1.285372  ]
 [ 0.03979941 -0.0671889   0.07435513]]
optimization_failed: False
expert_action: [[ 0.15221299 -0.9014953  -1.530576  ]
 [ 0.          0.          0.        ]]
optimization_failed: False
expert_action: [[-1.4358     0.4306963 -0.3983826]
 [ 0.         0.         0.       ]]
optimization_failed: True
expert_action: [[-0.02076995 -0.5209251   0.06268455]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[ 0.5375551  -0.6034368   0.03614739]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[ 0.72403544 -0.48639703 -0.34786612]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-1.1221437   1.2906765   1.3009112 ]
 [ 0.01678667 -0.02582353  0.0264991 ]]
optimization_failed: False
expert_action: [[-0.40601897  0.60681045  0.08583099]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.54156834  0.39741042  0

Exception in thread Thread-6081:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 86, in _plan_trajectory_thread
    _, joint_pos_traj = self.optimizer.optimize_trajectory(ee_traj, joint_pos, joint_vel, q_anchor)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/baseline/baseline_agent/optimizer.py", line 35, in optimize_trajectory
    success, dq_next = self._solve_aqp(des_point[:3], q_cur, dq_anchor)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/baseline/baseline_agent/optimizer.py", line 63, in _solve_aqp
    solver.setup(P=sparse.csc_matrix(P), q=q, A=sparse.csc_matrix(A), l=l, u=u, verbose=False,

expert_action: [[-0.75672007  0.6270686   1.2844229 ]
 [-0.35209808  0.74951625 -0.6568871 ]]
optimization_failed: False
expert_action: [[ 0.13669492 -0.07882933  0.22655794]
 [ 0.          0.          0.        ]]
optimization_failed: True
ERROR in validate_data: Lower bound at index 2 is greater than upper bound: 8.8230e+00 > -8.2098e-01
ERROR in osqp_setup: Problem data validation.
expert_action: [[ 0.12213133 -0.8665892  -2.1582603 ]
 [ 0.          0.          0.        ]]
optimization_failed: False
expert_action: [[-0.06129466  0.21133845 -0.2849607 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[ 0.83839035 -0.62171817 -1.1291972 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.3143282   0.20523943  0.9006417 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.11462268 -0.26904494 -0.629176  ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_acti

Exception in thread Thread-7011:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 85, in _plan_trajectory_thread
    ee_traj, hit_idx, q_anchor = self.plan_ee_trajectory(puck_pos, ee_pos)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 116, in plan_ee_trajectory
    res = np.array([self.bezier_planner.get_point(t_i) for t_i in np.arange(0, self.bezier_planner.t_final + 1e-6,
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent_wait.py", line 116, in <listcomp>
    res = np.array([self.bezier_planner.get_point(t_i) for t_i in np.ar

expert_action: [[-0.7499143   0.7784129   0.61364454]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.72624916  0.3337437  -0.3272969 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.8658305   0.81734294  1.1154131 ]
 [ 0.          0.          0.        ]]
optimization_failed: False
expert_action: [[ 0.18688375  0.07923381 -0.5553619 ]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-0.6656602   0.56562847  0.6301968 ]
 [-0.03941377  0.11725168 -0.09641077]]
optimization_failed: False
expert_action: [[-0.29255924  0.187323    0.45349982]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[ 0.14387763 -0.01725383 -0.15805583]
 [ 0.          0.          0.        ]]
optimization_failed: True
expert_action: [[-1.0143994  0.5898604 -0.3815038]
 [ 0.         0.         0.       ]]
optimization_failed: True
expert_action: [[-0.02091914  0.31819472  0.